In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.tree import DecisionTreeClassifier
from sklearn import tree
from sklearn.model_selection import train_test_split

movie_data = pd.read_csv('C:/Users/000110888/OneDrive - CSULB/Desktop/movie_data.csv')

code_gender = {'M': 1, 'F': 0}
code_member = {'yes': 1, 'no': 0}
code_rating = {'very bad': 1, 'bad': 2, 'okay': 3, 'good': 4, 'very good': 5}

movie_data['gender'] = movie_data['gender'].map(code_gender)
movie_data['member'] = movie_data['member'].map(code_member)
movie_data['rating'] = movie_data['rating'].map(code_rating)

X = movie_data.iloc[:, 0:4].values
y = movie_data.iloc[:, 4].values

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.20, random_state=687088
)

# ---------- helpers ----------
ALL_CLASSES = np.array([1, 2, 3, 4, 5])

def safe_div(num, den):
    return num / den if den != 0 else np.nan

def perf_measures(predclass, y_test, classes=ALL_CLASSES):
    predclass = np.asarray(predclass)
    y_test = np.asarray(y_test)

    tp = []
    fp = []
    tn = []
    fn = []

    for c in classes:
        tp_c = np.sum((predclass == c) & (y_test == c))
        fp_c = np.sum((predclass == c) & (y_test != c))
        tn_c = np.sum((predclass != c) & (y_test != c))
        fn_c = np.sum((predclass != c) & (y_test == c))
        tp.append(tp_c); fp.append(fp_c); tn.append(tn_c); fn.append(fn_c)

    tp = np.array(tp); fp = np.array(fp); tn = np.array(tn); fn = np.array(fn)
    N = len(y_test)

    print("CLASS MEASURES:")
    metrics = []
    for i, c in enumerate(classes):
        acc = safe_div(tp[i] + tn[i], N)
        mcr = safe_div(fp[i] + fn[i], N)
        sens = safe_div(tp[i], tp[i] + fn[i])          # recall
        fnr = safe_div(fn[i], tp[i] + fn[i])
        spec = safe_div(tn[i], fp[i] + tn[i])
        fpr = safe_div(fp[i], fp[i] + tn[i])
        prec = safe_div(tp[i], tp[i] + fp[i])          # handles "never predicted"
        npv = safe_div(tn[i], fn[i] + tn[i])
        f1 = safe_div(2 * tp[i], 2 * tp[i] + fn[i] + fp[i])

        metrics.append([acc, mcr, sens, fnr, spec, fpr, prec, npv, f1])

        print(f"\nCLASS: {c}")
        print("tp:", tp[i], "fp:", fp[i], "tn:", tn[i], "fn:", fn[i])
        print("accuracy:", acc)
        print("misclassrate:", mcr)
        print("sensitivity:", sens)
        print("FNR:", fnr)
        print("specificity:", spec)
        print("FPR:", fpr)
        print("precision:", prec)
        print("NPV:", npv)
        print("F1score:", f1)

    metrics = np.array(metrics)

    # Micro (your approach sums per-class confusion counts; OK for reporting)
    tp_sum, fp_sum, tn_sum, fn_sum = tp.sum(), fp.sum(), tn.sum(), fn.sum()

    print("\nMICRO MEASURES:")
    print("accuracy:", safe_div(tp_sum + tn_sum, tp_sum + fp_sum + tn_sum + fn_sum))
    print("misclassrate:", safe_div(fp_sum + fn_sum, tp_sum + fp_sum + tn_sum + fn_sum))
    print("sensitivity:", safe_div(tp_sum, tp_sum + fn_sum))
    print("FNR:", safe_div(fn_sum, tp_sum + fn_sum))
    print("specificity:", safe_div(tn_sum, fp_sum + tn_sum))
    print("FPR:", safe_div(fp_sum, fp_sum + tn_sum))
    print("precision:", safe_div(tp_sum, tp_sum + fp_sum))
    print("NPV:", safe_div(tn_sum, fn_sum + tn_sum))
    print("F1-score:", safe_div(2 * tp_sum, 2 * tp_sum + fn_sum + fp_sum))

    # Macro: mean ignoring NaN (important when class absent in truth/pred)
    print("\nMACRO MEASURES:")
    macro = np.nanmean(metrics, axis=0)
    labels = ["accuracy","misclassrate","sensitivity","FNR","specificity","FPR","precision","NPV","F1score"]
    for name, val in zip(labels, macro):
        print(f"{name}:", val)

    # Weighted macro: weights are true supports normalized to sum to 1
    support = tp + fn
    w = support / support.sum() if support.sum() > 0 else np.zeros_like(support, dtype=float)

    print("\nWEIGHTED MACRO MEASURES:")
    wmacro = np.nansum(metrics * w[:, None], axis=0)
    for name, val in zip(labels, wmacro):
        print(f"{name}:", val)

# ---------- GINI tree ----------
gini_tree = DecisionTreeClassifier(max_leaf_nodes=6, criterion='gini', random_state=707720)
gini_tree.fit(X_train, y_train)

plt.figure(figsize=(15, 10))
tree.plot_tree(gini_tree, feature_names=['age','gender','member','nmovies'], filled=True)
plt.show()

proba = gini_tree.predict_proba(X_test)
predclass = gini_tree.classes_[np.argmax(proba, axis=1)]

print("\nPERFORMANCE MEASURES FOR FITTED GINI TREE")
perf_measures(predclass, y_test)

# ---------- ENTROPY tree ----------
entropy_tree = DecisionTreeClassifier(max_leaf_nodes=6, criterion='entropy', random_state=707720)
entropy_tree.fit(X_train, y_train)

plt.figure(figsize=(15, 10))
tree.plot_tree(entropy_tree, feature_names=['age','gender','member','nmovies'], filled=True)
plt.show()

proba = entropy_tree.predict_proba(X_test)
predclass = entropy_tree.classes_[np.argmax(proba, axis=1)]

print("\nPERFORMANCE MEASURES FOR FITTED ENTROPY TREE")
perf_measures(predclass, y_test)

#########################################################################

#FITTING MULTINOMIAL CLASSIFICATION TREE WITH CHAID SPLITTING CRITERION (ORDINAL CHAID TREE)
from chefboost import Chefboost

movie_data=pandas.read_csv('C:/Users/000110888/OneDrive - CSULB/Desktop/movie_data.csv')
code_gender={'M':1,'F':0}
code_member={'yes':1,'no':0}
#Note: rating is used as nominal

movie_data['gender']=movie_data['gender'].map(code_gender)
movie_data['member']=movie_data['member'].map(code_member)

X=movie_data.iloc[:,0:4].values
y=movie_data.iloc[:,4].values

#SPLITTING DATA INTO 80% TRAINING AND 20% TESTING SETS
X_train, X_test, y_train, y_test=train_test_split(X, y, test_size=0.20, 
random_state=687088)

X_train=pandas.DataFrame(X_train, columns=['age','gender','member','nmovies'])
y_train=pandas.DataFrame(y_train, columns=['rating'])
train_data=pandas.concat([X_train, y_train], axis=1) #one-to-one concatenation

#FITTING BINARY TREE WITH CHAID SPLITTING ALGORITHM
config={'algorithm': 'CHAID', 'max_depth': 3}
tree_chaid=Chefboost.fit(train_data, config, target_label='rating')

#COMPUTING PREDICTION ACCURACY
X_test=pandas.DataFrame(X_test, columns=['age','gender','member','nmovies'])

y_pred=[]
for i in range(len(y_test)):
    y_pred.append(Chefboost.predict(tree_chaid, X_test.iloc[i,:].to_numpy()))
predclass=numpy.asarray(y_pred)


#COMPUTING PERFORMANCE MEASURES FOR FITTED CHAID TREE
#turning rating into ordinal to use perf_measures function
code_rating={'very bad':1,'bad':2,'okay':3,'good':4,'very good':5}
df=pandas.DataFrame({'y_test': y_test,'predclass': predclass})
df['y_test']=df['y_test'].map(code_rating)
df['predclass']=df['predclass'].map(code_rating)
y_test_ord=df['y_test']
predclass_ord=df['predclass']

print()
print('PERFORMANCE MEASURES FOR FITTED CHAID TREE')
perf_measures(predclass_ord, y_test_ord)
